# Цель лабораторной работы
Изучить сложные способы подготовки выборки и подбора гиперпараметров на примере метода ближайших соседей.

# Задание
Требуется выполнить следующие действия:

1. Выбрать набор данных (датасет) для решения задачи классификации или регресии.
2. В случае необходимости проведите удаление или заполнение пропусков и кодирование категориальных признаков.
3. С использованием метода train_test_split разделите выборку на обучающую и тестовую.
4. Обучите модель ближайших соседей для произвольно заданного гиперпараметра 
5. Оцените качество модели с помощью трех подходящих для задачи метрик.
6. Постройте модель и оцените качество модели с использованием кросс-валидации. Проведите эксперименты с тремя различными стратегиями кросс-валидации.
7. Произведите подбор гиперпараметра с использованием GridSearchCV и кросс-валидации.
8. Сравните качество полученной модели с качеством модели, полученной в пункте 4.

# Ход выполнения работы
Подключим все необходимые библиотеки и настроим отображение графиков:

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import * 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style ="ticks")


from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler, Normalizer


In [2]:
df = pd.read_csv('country_wise_latest.csv')

## Предварительная подготовка данных

In [3]:
data = df.copy()

In [4]:
df.head()

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa


In [5]:
#Кодирование категориальных признаков

df["WHO Region"] = df["WHO Region"].astype('category')
df["Country/Region"] = df["Country/Region"].astype('category')

#Назначить закодированную переменную новосу столбцу с помощью метода доступа
df["WHO Region_cat"] = df["WHO Region"].cat.codes
df["Country/Regioncat"] = df["Country/Region"].cat.codes


In [6]:
df = df.drop(['WHO Region', 'Country/Region', 'Deaths / 100 Recovered'], axis=1)

In [7]:
df.head()

,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Confirmed last week,1 week change,1 week % increase,WHO Region_cat,Country/Regioncat
0,36263,1269,25198,9796,106,10,18,3.50,69.49,35526,737,2.07,2,0
1,4880,144,2745,1991,117,6,63,2.95,56.25,4171,709,17.00,3,1
2,27973,1163,18837,7973,616,8,749,4.16,67.34,23691,4282,18.07,0,2
3,907,52,803,52,10,0,0,5.73,88.53,884,23,2.60,3,3
4,950,41,242,667,18,1,0,4.32,25.47,749,201,26.84,0,4


In [8]:
df.isna().sum()

Confirmed                0
Deaths                   0
Recovered                0
Active                   0
New cases                0
New deaths               0
New recovered            0
Deaths / 100 Cases       0
Recovered / 100 Cases    0
Confirmed last week      0
1 week change            0
1 week % increase        0
WHO Region_cat           0
Country/Regioncat        0
dtype: int64

In [9]:
df = df.dropna(axis=0)

In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Confirmed,187.0,88130.935829,383318.663831,10.00,1114.000,5059.00,40460.500,4290259.00
Deaths,187.0,3497.518717,14100.002482,0.00,18.500,108.00,734.000,148011.00
Recovered,187.0,50631.481283,190188.189643,0.00,626.500,2815.00,22606.000,1846641.00
Active,187.0,34001.935829,213326.173371,0.00,141.500,1600.00,9149.000,2816444.00
New cases,187.0,1222.957219,5710.374790,0.00,4.000,49.00,419.500,56336.00
New deaths,187.0,28.957219,120.037173,0.00,0.000,1.00,6.000,1076.00
New recovered,187.0,933.812834,4197.719635,0.00,0.000,22.00,221.000,33728.00
Deaths / 100 Cases,187.0,3.019519,3.454302,0.00,0.945,2.15,3.875,28.56
Recovered / 100 Cases,187.0,64.820535,26.287694,0.00,48.770,71.32,86.885,100.00
Confirmed last week,187.0,78682.475936,338273.676567,10.00,1051.500,5020.00,37080.500,3834677.00


## Разделение данных
Разделим данные на целевой столбец и признаки:

При построении предсказательных моделей исходные данные обычно разбиваются на обучающую ("training set") и контрольную ("test set") выборки. 
**Обучающая выборка** используется для построения математических отношений между некоторой переменной-откликом и предикторами, тогда как **контрольная (= "проверочная")** выборка служит для получения оценки прогнозных свойств модели на новых данных, т.е. данных, которые не были использованы для обучения модели.
В нашем случае обучающая выборка - это Customers_Rated, а проверочная - это все остальные признаки, которые потенциально могут влиять на решение клиента

In [11]:
y = df['WHO Region_cat']  #Наименования признаков
X = df.drop('WHO Region_cat', axis=1) # Значения признаков

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state= 45)
# random_state позволяет задавать базовое значение для генератора случайных чисел, чтобы сделать выборку неслучайной 

In [13]:
# Размер обучающей выборки
X_train.shape, y_train.shape

((140, 13), (140,))

In [14]:
# Размер тестовой выборки
X_test.shape, y_test.shape

((47, 13), (47,))

In [15]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5], dtype=int8)

In [16]:
np.unique(y_test)

array([0, 1, 2, 3, 4, 5], dtype=int8)

## Модель ближайших соседей для произвольно заданного гиперпараметра *K*
Напишем функцию, которая считает метрики построенной модели:

In [17]:
#Масштабирование данных
scaler = MinMaxScaler().fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_train.columns)
X_train.describe()

,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Confirmed last week,1 week change,1 week % increase,Country/Regioncat
count,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000
mean,0.024663,0.026513,0.033277,0.014361,0.027061,0.033696,0.034996,0.110149,0.658503,0.024485,0.026264,0.086341,0.492355
std,0.102483,0.105583,0.117467,0.087061,0.116545,0.127919,0.143011,0.125706,0.258571,0.101104,0.119959,0.106582,0.287452
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000268,0.000145,0.000354,0.000046,0.000084,0.000000,0.000000,0.033176,0.510100,0.000278,0.000215,0.033669,0.252703
50%,0.001724,0.000888,0.002101,0.000608,0.001172,0.000929,0.000904,0.078957,0.722700,0.001789,0.001188,0.055487,0.489189
75%,0.010953,0.007452,0.014351,0.003843,0.008361,0.005576,0.007790,0.145746,0.878550,0.011075,0.008507,0.108294,0.731081
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [18]:
def test_model(model):
    print("mean_absolute_error:",
          mean_absolute_error(y_test, model.predict(X_test)))
    print("mean_squared_error:",
          mean_squared_error(y_test, model.predict(X_test)))
    print("median_absolute_error:",
          median_absolute_error(y_test, model.predict(X_test)))
    print("r2_score:",
          r2_score(y_test, model.predict(X_test)))

Попробуем метод ближайших соседей с гиперпараметром K = 10:

In [19]:
reg_10 = KNeighborsRegressor(n_neighbors=10)
reg_10.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=10)

In [20]:
test_model(reg_10)

mean_absolute_error: 1.4638297872340424
mean_squared_error: 2.862978723404256
median_absolute_error: 1.5
r2_score: -0.1419862766341642


1) mean_absolute_error: 1.463, чем ближе значение к нулю, тем лучше качество регрессии.

2) mean_squared_error: 2.862, чем ближе значение к нулю, тем лучше модель

3) median_absolute_error: 1.5

4) r2_score: -0.14, чем ближе значение коэффициента к 0, тем слабее зависимость, то есть разброс предсказаний модели  𝑦̂   относительно разброса самой целевой переменной  𝑦  ) достаточно велик - то есть модель плохо описывает данные.